In [2]:
import pandas as pd
import folium
from docplex.mp.model import Model
import docplex.mp.solution as Solution
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

$$
\text{The problem is formulated with the sequential formulation of Miller, Tucker, Zemlin (1960) (MTZ). They define a new} 
$$

$$
\text{decision variable } u_i \text{ to indicate the sequence in which city i is visited (i = 2, · · · , n)}
$$


$$
\sum_{i=1}^{n} \sum_{j=1}^{n} x_{ij}d_{ij}
$$

$$
s.t.
$$

$$
\sum_{i: i \neq j } x_{ij} = 1 \quad \forall i
$$


$$
\sum_{j: j \neq i } x_{ij} = 1 \quad \forall j
$$


$$
u_{i} - u_{j} + n \cdot (1 - x_{ij}) \leq n - 2, \quad i \neq j, i, j = 2, 3, 4, ..., n
$$

$$
x_{ij} \in \{0, 1\} \quad \forall i, j
$$


$$
\text{To solve the problem, the solver IBM ILOG CPLEX Optimization Studio has been utilized.}
$$

In [11]:
df = pd.read_excel("TSP Dataset/Dataset 1.xlsx", sheet_name=0)
df = df.set_index('İLÇE ADI') 
df.index.names = [None] 
df=df.fillna(0)

In [3]:
df

,ARNAVUTKÖY,ATAŞEHİR,AVCILAR,BAĞCILAR,BAHÇELİEVLER,BAKIRKÖY,BAŞAKŞEHİR,BAYRAMPAŞA,BEŞİKTAŞ,BEYKOZ,...,SARIYER,SİLİVRİ,SULTANBEYLİ,SULTANGAZİ,ŞİLE,ŞİŞLİ,TUZLA,ÜMRANİYE,ÜSKÜDAR,ZEYTİNBURNU
ARNAVUTKÖY,0.000000,59.039768,67.805428,49.361971,61.770984,62.144750,50.899064,47.363597,46.329528,74.794273,...,51.503548,79.869694,78.911457,44.459830,116.062560,45.033637,82.476783,55.421857,50.096338,51.050957
ATAŞEHİR,59.039768,0.000000,43.634007,29.780762,29.771968,29.398202,37.125158,23.925698,12.710240,21.815769,...,28.290340,86.735195,22.656558,30.685924,63.084056,16.329084,26.221884,6.972615,8.943430,27.613058
AVCILAR,67.805428,43.634007,0.000000,22.148155,13.862039,14.235805,18.506364,19.708309,30.923767,59.388512,...,46.495302,43.101188,63.505696,28.405598,100.656799,27.304923,67.071022,40.016096,34.690577,16.020949
BAĞCILAR,49.361971,29.780762,22.148155,0.000000,16.113710,16.487476,5.241791,7.000239,17.070522,45.535267,...,28.051844,65.249343,49.652451,9.962141,86.803554,13.451678,53.217777,26.162851,20.837332,18.272620
BAHÇELİEVLER,61.770984,29.771968,13.862039,16.113710,0.000000,0.373766,12.471919,5.846270,17.061727,45.526473,...,30.175748,56.963228,49.643656,22.371154,86.794760,13.442884,53.208983,26.154056,20.828538,2.158910
BAKIRKÖY,62.144750,29.398202,14.235805,16.487476,0.373766,0.000000,12.845685,5.472504,16.687962,45.152707,...,29.801982,57.336993,49.269891,22.744919,86.420994,13.069118,52.835217,25.780291,20.454772,1.785144
BAŞAKŞEHİR,50.899064,37.125158,18.506364,5.241791,12.471919,12.845685,0.000000,18.318189,24.414917,52.879663,...,29.588938,61.607552,56.996847,11.499234,94.147950,25.914803,60.562173,33.507247,28.181728,14.630830
BAYRAMPAŞA,47.363597,23.925698,19.708309,7.000239,5.846270,5.472504,18.318189,0.000000,11.215457,39.680203,...,24.329478,62.809498,43.797387,13.295474,80.948490,7.596614,47.362713,20.307787,14.982268,3.687360
BEŞİKTAŞ,46.329528,12.710240,30.923767,17.070522,17.061727,16.687962,24.414917,11.215457,0.000000,28.464746,...,15.580099,74.024955,32.581929,17.975683,69.733032,3.618844,36.147256,9.092329,3.766810,14.902817
BEYKOZ,74.794273,21.815769,59.388512,45.535267,45.526473,45.152707,52.879663,39.680203,28.464746,0.000000,...,44.044845,102.489701,44.472327,46.440429,75.483857,32.083589,48.037654,19.372416,24.697935,43.367563


In [4]:
n = 15
cities =[i for i in range(n)] 
edges =[(i,j) for i in cities for j in cities if i!=j]

In [5]:
m=Model('TSP')

In [6]:
x = m.binary_var_dict(edges, name = 'x')
u = m.continuous_var_dict(cities, name ='u')

In [7]:
distances ={(i, j): df.iloc[i, j] for i,j in edges}

In [8]:
m.minimize(m.sum(distances[e]*x[e] for e in edges))

In [9]:
# Constraint 1: each city must be entered once
for c in cities:
    if c != 0:
        m.add_constraint(m.sum(x[(i,j)] for i,j in edges if i==c)==1, ctname=f'city_out__{c}')

# Constraint 2: each city must be exited once
for c in cities:
    if c != 0:
        m.add_constraint(m.sum(x[(i,j)] for i,j in edges if j==c)==1, ctname=f'city_in_{c}')

In [10]:
# Constraint 3: ensures that u_j = u_i + 1 if and only if x_ij = 1
for i,j in edges:
    if j!=0:
        m.add_indicator(x[(i,j)],u[(i)]+1==u[(j)], name='order_{i}_{j}')

In [11]:
# m.parameters.timelimit=600
# m.parameters.mip.strategy.branch=1
# m.parameters.mip.tolerances.mipgap=0.15

solution = m.solve(log_output=True)

Version identifier: 22.1.1.0 | 2023-06-15 | d64d5bd77
CPXPARAM_Read_DataCheck                          1
Tried aggregator 2 times.
MIP Presolve modified 91 coefficients.
Aggregator did 91 substitutions.
Reduced MIP has 133 rows, 330 columns, and 707 nonzeros.
Reduced MIP has 210 binaries, 0 generals, 0 SOSs, and 196 indicators.
Presolve time = 0.02 sec. (0.62 ticks)
Found incumbent of value 1548.946714 after 0.03 sec. (0.91 ticks)
Probing time = 0.00 sec. (0.32 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 133 rows, 330 columns, and 707 nonzeros.
Reduced MIP has 210 binaries, 0 generals, 0 SOSs, and 196 indicators.
Presolve time = 0.02 sec. (0.47 ticks)
Probing time = 0.00 sec. (0.32 ticks)
Clique table members: 119.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution time = 0.00 sec. (0.23 ticks)

        Nodes                                        

In [12]:
solution.display()

solution for: TSP
objective: 237.757
status: OPTIMAL_SOLUTION(2)
x_0_13 = 1
x_1_0 = 1
x_2_4 = 1
x_3_7 = 1
x_4_5 = 1
x_5_6 = 1
x_6_3 = 1
x_7_11 = 1
x_8_14 = 1
x_9_1 = 1
x_10_2 = 1
x_11_8 = 1
x_12_10 = 1
x_13_12 = 1
x_14_9 = 1
u_1 = 14.000
u_2 = 4.000
u_3 = 8.000
u_4 = 5.000
u_5 = 6.000
u_6 = 7.000
u_7 = 9.000
u_8 = 11.000
u_9 = 13.000
u_10 = 3.000
u_11 = 10.000
u_12 = 2.000
u_13 = 1.000
u_14 = 12.000


In [13]:
lst_c = []
for c in u:
    soln_c = (c,int(solution.get_var_value(u[c])))
    lst_c.append(soln_c)
df_c = pd.DataFrame.from_records(lst_c, columns = ['city', 'visit order'])
df_c.sort_values(by=['visit order'], inplace = True)
df_c

,city,visit order
0,0,0
13,13,1
12,12,2
10,10,3
2,2,4
4,4,5
5,5,6
6,6,7
3,3,8
7,7,9


In [14]:
connection_order = []

connection_order = [i for i in df_c["city"]]
connection_order.append(0)

In [16]:
def coordinates(district):
    
    geolocator = Nominatim(user_agent="tsp_visualization")

    if district == "ARNAVUTKÖY":
        return (41.186398, 28.738869)
    else:
        location = geolocator.geocode(f"{district}, Istanbul, Turkey")
        return (location.latitude, location.longitude)

# Get coordinates for all districts
coordinates = {district: coordinates(district) for district in df.index}

In [19]:
def mapper(order, dataframe, coordinates):

    geolocator = Nominatim(user_agent="tsp_visualization")

    # Create a folium map centered around Istanbul
    istanbul = geolocator.geocode("Istanbul, Turkey")
    m = folium.Map(location=(istanbul.latitude, istanbul.longitude), zoom_start=13)

    # Map the district names to their indices
    district_indices = {i: district for i, district in enumerate(dataframe.index)}

   # Add numbered markers for each district in district_indices
    for index, district_name in enumerate(dataframe.index):
        coord = coordinates.get(district_name)
        if coord:
            folium.Marker(
                location=coord,
                popup=f"{index}: {district_name}",
                tooltip=f"{index}: {district_name}",
                icon=folium.DivIcon(
                    html=f"""
                    <div style="
                        font-size: 14pt;
                        font-weight: bold;
                        color: white;
                        background-color: black;
                        border-radius: 50%;
                        width: 24px;
                        height: 24px;
                        text-align: center;
                        line-height: 24px;
                    ">{index}</div>
                    """
                )
            ).add_to(m)

     # Add lines connecting the districts in the given order
        for i in range(len(order)):
            if i != len(order) - 1:
                folium.PolyLine(locations=[coordinates[district_indices[order[i]]], coordinates[district_indices[order[i+1]]]], color='blue').add_to(m)
    return m


$$
\text{For the interactive map:}
$$

<div style="text-align: center;">
    <a href="https://nbviewer.org/github/ali-yildirim/Maps/blob/main/Maps.ipynb">https://nbviewer.org/github/ali-yildirim/Maps/blob/main/Maps.ipynb</a>
</div>


In [4]:
connection_order = [0, 13, 12, 10, 2, 4, 5, 6, 3, 15, 7, 11, 8, 9, 14, 1, 0]


In [24]:
connection_order_2 = [0, 13, 29, 12, 10, 16, 2, 24, 4, 5, 20, 37, 7, 18, 17, 15, 3, 6, 31, 19, 22, 28, 35, 9, 14, 27, 32, 34, 26, 30, 23, 25, 1, 21, 36, 8, 33, 11, 0]